## Coursera - Deep Learning with Keras and Tensorflow
### Module 1

# Sequential API VS. Functional API


## Sequential

### Imports

In [ ]:
from Tensorflow.keras.models import Sequential
from Tensorflow.kersas.layers import Dense

In [ ]:
# Create a sequential model

model = Sequential([
    Dense(64, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Functional

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# Define the input
inputs = Input(shape=(784,))

In [ ]:
# Define layers
x = Dense(64, activation='relu')(inputs)
outputs = Dense(10, activation='softmax')(x)

In [ ]:
# create the model
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## complex model with multiple inputs

In [ ]:
from tensorflow.keras.layers import concatenate

In [ ]:
# define two sets of inputs
inputA = Input(shape=(64,))
inputB = Input(shape=(128,))

In [ ]:
# the first branch operates in the first input
x = Dense(16, activation='relu')(inputA)
x = Dense(8, activation='relu')(x)

In [ ]:
# the second branch operates on the second input
y = Dense(64, activation='relu')(inputB)
y = Dense(32, activation='relu')(y)

In [ ]:
# combine the output of the two branches
combined = concatenate([x, y])

In [ ]:
# apply a dense layer then a regression prediction on the combined outputs
z = Dense(2, activation='relu')(combined)
z = Dense(1, activation='linear')(z)

In [ ]:
# the model will accept the inputs of the two branches and output a single value
model = Model(inputs=[inputA, inputB], outputs=z)

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

## Shared layers

helpful when applying the same transformation to multiple inputs

you can use shared layers to process 2 different inputs through the same layers and then compare their outputs

In [ ]:
from tensorflow.keras.layers import lambda

In [ ]:
# define the input layer
input = Input(shape=(28,28,1))

In [ ]:
# define a shared convolutional base
conv_base = Dense(64, activation='relu')

In [ ]:
# process the input through the shared layer
processed_1 = conv_base(input)
processed_2 = conv_base(input)

In [ ]:
# create a model using the shared layer

model = Model(inputs=input, outputs=[processed_1, processed_2])

# practical example : Implementing a complex model

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.activations import relu, softmax

In [ ]:
# first input model
inputA = Input(shape=(32,32,1))
x = Conv2D(32, (3,3), activation=relu)(inputA)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Model(inputs=inputA, outputs=x)
# second input model
inputB = Input(shape=(32,32,1))
y = Conv2D(32, (3,3), activation=relu)(inputB)
y = MaxPooling2D((2,2))(y)
y = Flatten()(y)
y = Model(inputs=inputB, outputs=y)

In [ ]:
# combine the output of the two branches
combined = concatenate([x.output, y.output])

In [ ]:
# apply a FC layer and then a regression prediction on the combined outputs
z = Dense(64, activation=relu)(combined)
z = Dense(1, activation=softmax)(z)

In [ ]:
# the model will accept the inputs of the two branches and then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

# Subclassing API

offers the most flexibility

- allows to model custom and dynamic models by subclassing the model class and implementing your own call() method

- particularly useful when you need to build models where the forward pass cannot be defined statistically

-- we need to subclass the model class and define our layers in the init method



-- then we implement the forward pass in the call() method where we explicitly define how the layers are connected and how the data flow through the model

In [ ]:
import tensorflow as tf

In [ ]:
# define the model by sybclassing

class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    # define layers
    self.dense1 = tf.keras.layers.Dense(64, activation='relu')
    self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, inputs):
    # define the forward pass
    x = self.dense1(inputs)
    return self.dense2(x)


In [ ]:
# instantiate the model
model = MyModel()

In [ ]:
# define loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## Use cases for the subclassing API

- models with dynamic architecture. when the architecture of the model needs to be changed dynamically such as certain types of reinforcement learning

- custom training loops, when you need more control over the training process

- for experimenting with new types of layers or architectures that are not yet available in the the standard keras API


- allows the use of dynamic graphs, which means you can change the architecture of the model dynamically during training based on the input data or conditions
while in the functional or sequential APIs static graphs are used

# implementing a custom training loop

- using tf.GradientTape method

- provides more flexibility and control over the training process compared to the built in keras.fit method

In [ ]:
epochs = 5

# create a dummy training dataset
(train_images, train_labels), _ = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28*28).astype('float32') / 255

# Flatten and normalize
train_labels = train_labels.astype('float32')

# create a tf.data dataset for batching
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)

# custom training loop
for epoch in range(epochs):
  print(f'epoch {epoch+1}/{epochs}')
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      # forward pass
      y_pred = model(x_batch, training = True)
      # compute loss
      loss = loss_fn(y_batch, y_pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'loss: {loss.numpy()}')

In [ ]:
!pip install networkx
!pip install matplotlib
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# create a graph
G = nx.DiGraph()

# adding nodes
G.add_node('input')
G.add_node('Condition check')
G.add_node('Path 1 Layer 1')
G.add_node('Path 1 Layer 2')
G.add_node('Path 2 Layer 1')
G.add_node('Path 2 Layer 2')
G.add_node('output')

# Adding edges for dynamic flow

G.add_edges_from([
    ('input', 'Condition check'),
    ('Condition check', 'Path 1 Layer 1'),
    ('Path 1 Layer 1', 'Path 1 Layer 2'),
    ('Path 1 Layer 2', 'output'),
    ('Condition check', 'Path 2 Layer 1'),
    ('Path 2 Layer 1', 'Path 2 Layer 2'),
    ('Path 2 Layer 2', 'output') ])


# position nodels using a shell layout
pos = nx.shell_layout(G)

# draw the graph
plt.figure(figsize=(8, 6))
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=10, font_color='black', edge_color='gray')
plt.title('Dynamic Graph Visualization')
plt.show()